In [115]:
import json
import random
from eval import Evaluate
import torch
eval_f = Evaluate()

In [116]:
def get_original_samples(path, is_unk):
    abstracts = {}
    with open(path) as f:
        for line in f:
            j = json.loads(line.strip())
            if not is_unk:
                abstracts[j["title"]] = j["abstract"]
            else:
                abstracts[j["abstract"]] = j["title"]
    return abstracts        

In [117]:
def get_generated_samples(path):
    abstracts = {}
    with open(path) as f:
        for line in f:
            j = json.loads(line.strip())
            abstracts[j["original"]] = j["generated"]
    return abstracts       

In [118]:
def get_abstract_metrics(gen, org):
    ref = {0 : [org]}
    cand = {0 : gen}
    final_scores = eval_f.evaluate(live=True, cand=cand, ref=ref)
    return final_scores

In [149]:
original_abstracts = get_original_samples("arxiv-dataset/arxiv-original.txt", is_unk=False)
original_unk = get_original_samples("arxiv-dataset/arxiv-UNK-abstracts.txt", is_unk=True)
abstract1 = get_generated_samples("arxiv-dataset/WEPGen/Arxiv-WE-Topics-Structure-IntraAttention-Polishing-BLEU-4=3.7366467606952156.txt")
abstract2 = get_generated_samples("arxiv-dataset/WEPGen/Arxiv-WE-Topics-Structure-IntraAttention-Polishing-BLEU-4=3.754111447858121.txt")

In [150]:
ratings = []
c = 0
for key in abstract1.keys():
    original_abs_UNK = key
    title = original_unk[original_abs_UNK]
    org = original_abstracts[title]
    gen1 = abstract1[key]
    gen2 = abstract2[key]
    metrics = get_abstract_metrics(gen2, org)
    ratings.append((title, org, gen1, gen2, metrics.values()))
    c += 1
    
    if c % 200 == 0:
        print("Done {}".format(c))

Done 200
Done 400
Done 600
Done 800
Done 1000


In [151]:
bleu_samples = sorted(ratings, reverse=True, key=lambda x: sum(list(x[-1])[:4]))[:100]
meteor_samples = sorted(ratings, reverse=True, key=lambda x: list(x[-1])[4])[:100]
rouge_samples = sorted(ratings, reverse=True, key=lambda x: list(x[-1])[5])[:100]
combined_samples = sorted(ratings, reverse=True, key=lambda x: sum(x[-1]))[:100]

In [152]:
bleu_set = set([b[:4] for b in bleu_samples])
meteor_set = set([b[:4] for b in meteor_samples])
rouge_set = set([b[:4] for b in rouge_samples])
combined_set = set([b[:4] for b in combined_samples])

In [153]:
A = bleu_set.intersection(meteor_set)
B = A.intersection(rouge_set)
final_set = list(B.intersection(combined_set))

In [154]:
print(len(final_set))

24


In [155]:
qualtrics_final_same_title = {}
i = 0
for title, original, gen1, gen2 in final_set:
    qualtrics_final_same_title[i] = [title, (original, "Original"), (gen1, "WEPGen"), (gen2, "WEPGen")]
    i += 1

In [156]:
with open("arxiv-dataset/Turing/Block-1/B1-W.txt", "w") as f:
    f.write(json.dumps(qualtrics_final_same_title))

In [157]:
qualtrics_final_different_title = {}
titles = list(original_abstracts.keys())
i = 0
for title, original, gen1, gen2 in final_set:
    
    # First original abstract
    org_title1 = random.choice(titles)
    org_abstract1 = original_abstracts[org_title1]
    
    # Second original abstract
    org_title2 = random.choice(titles)
    org_abstract2 = original_abstracts[org_title2]
    
    # Fake abstract
    fake_title = title
    fake_abstract = gen2
    
    qualtrics_final_different_title[i] = [(org_title1, org_abstract1, "Original"), (org_title2, org_abstract2, "Original"), (fake_title, fake_abstract, "WEPGen")]
    i += 1
    
with open("arxiv-dataset/Turing/Block-2/B2-W.txt", "w") as f:  
    f.write(json.dumps(qualtrics_final_different_title))